In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In [2]:
using Pkg
Pkg.activate("../..")
Pkg.status()

  Activating 

      Status

new environment at `c:\Project.toml`


 `C:\Project.toml` (empty project)


# Algorithms

* Algorithm is loosely defined as a set of instructions for doing something, which terminates in finite time. An algorithm requires input and output.

* [Donald Knuth](https://en.wikipedia.org/wiki/The_Art_of_Computer_Programming): (1) finiteness, (2) definiteness, (3) input, (4) output, (5) effectiveness. [Five things that Algorithm should have]


## Measure of efficiency

* A basic unit for measuring algorithmic efficiency is **flop**.  
> A flop (**floating point operation**) consists of a floating point addition, subtraction, multiplication, division, or comparison, and the usually accompanying fetch and store.  

Some books count multiplication followed by an addition (fused multiply-add, FMA) as one flop. This results a factor of up to 2 difference in flop counts.

* How to measure efficiency of an algorithm? Big O notation. If $n$ is the size of a problem, an algorithm has order $O(f(n))$, where the leading term in the number of flops is $c \cdot f(n)$. For example,
    - matrix-vector multiplication `A * b`, where `A` is $m \times n$ and `b` is $n \times 1$, takes $2mn$ or $O(mn)$ flops
        - $m$ dot products of $n$-dimensional vectors. dot product of $n$-dimensional vectors require $n$ multiplications and $n$ (or exactly $n-1$) additions.  
    - matrix-matrix multiplication `A * B`, where `A` is $m \times n$ and `B` is $n \times p$, takes $2mnp$ or $O(mnp)$ flops
    - If `A`, `B` are $n\times n$ dimensional matrices and `b` is $n$-dimensional vector then `A * b` takes $O(n^2)$ flops and `A * B` takes $O(n^3)$ flops.

* A hierarchy of computational complexity:  
    Let $n$ be the problem size.
    - Exponential order: $O(b^n)$ ("horrible" ; as $n$ grows by 1, required flops increases to b times its value.)    
    - Polynomial order: $O(n^q)$ (doable)  
    - $O(n \log n )$ (fast)  
    - Linear order $O(n)$ (fast)  
    - Logarithmic order $O(\log n)$ (super fast)  
    
* Classification of data sets by [Huber](http://link.springer.com/chapter/10.1007%2F978-3-642-52463-9_1) (1994).

| Data Size | Bytes     | Storage Mode          |
|-----------|-----------|-----------------------|
| Tiny      | $10^2$    | Piece of paper        |
| Small     | $10^4$    | A few pieces of paper |
| Medium    | $10^6$ (megabytes)    | A floppy disk         |
| Large     | $10^8$    | Hard disk             |
| Huge      | $10^9$ (gigabytes)   | Hard disk(s)          |
| Massive   | $10^{12}$ (terabytes) | RAID storage          |

In today's bigdata age, notion for data size shifts to right about $10^3$.

* Difference of $O(n^2)$ and $O(n\log n)$ on massive data. Suppose we have a teraflops supercomputer capable of doing $10^{12}$ flops per second. For a problem of size $n=10^{12}$, $O(n \log n)$ algorithm takes about 
$$10^{12} \log (10^{12}) / 10^{12} \approx 27 \text{ seconds}.$$ 
$O(n^2)$ algorithm takes about $10^{24}/10^{12} = 10^{12}$ seconds, which is approximately 31710 years! (Problem is, $n=10^{12}$ is not unrealistically large problem size in these days. In fact, it is quite common problem size nowadays.)

* QuickSort and Fast Fourier Transform (invented by John Tukey) are celebrated algorithms that turn $O(n^2)$ operations into $O(n \log n)$. Another example is the Strassen's method for matrix multiplication, which turns $O(n^3)$ matrix multiplication into $O(n^{\log_2 7})$.
    - Past algorithms for sorting and FFT typically had order $O(n^2)$. QuickSort algorithm is known to be optimal one. Tukey invented algorithm for FFT having order $O(n\log n)$.
    - Naive implementation for matrix multiplication has order $O(n^3)$. Strassen's method attains order $O(n^{\log_2 7})$ where $2<\log_2 7<3$.    

* One goal of this course is to get familiar with the flop counts for some common numerical tasks in statistics.   
> **The form of a mathematical expression and the way the expression should be evaluated in actual practice may be quite different.**
\
    -- James Gentle, *Matrix Algebra*, Springer, New York (2007).


* For example, compare flops of the two mathematically equivalent expressions: `A * B * x` and `A * (B * x)` where `A` and `B` are matrices and `x` is a vector.


In [4]:
using BenchmarkTools, Random

Random.seed!(123) # seed
n = 1000
A = randn(n, n)
B = randn(n, n)
x = randn(n)

# complexity is n^3 + n^2 = O(n^3)
@benchmark $A * $B * $x

BenchmarkTools.Trial: 269 samples with 1 evaluation.
 Range (min … max):  14.510 ms … 97.294 ms  ┊ GC (min … max): 0.00% … 80.63%
 Time  (median):     18.157 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.627 ms ±  6.956 ms  ┊ GC (mean ± σ):  3.13% ±  6.96%

             ▃▃     ▁ ▁    ▁▃▃▅█▁▂▁ ▃                          
  ▃▁▁▃▁▁▄▁▃▇▆██▅▆▇███▆█▆█▄███████████▅▇▇▄█▄▅▃▃▃▁▄▃▁▃▃▁▄▃▃▃▃▁▃ ▄
  14.5 ms         Histogram: frequency by time        22.4 ms <

 Memory estimate: 7.64 MiB, allocs estimate: 3.

In [5]:
# complexity is n^2 + n^2 = O(n^2)
@benchmark $A * ($B * $x)

BenchmarkTools.Trial: 7629 samples with 1 evaluation.
 Range (min … max):  496.037 μs …   3.428 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     542.873 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   639.769 μs ± 224.916 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▆▅▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁                                     ▂
  ██████████████████████████████▇█▇▇▇▇▇▆▇▇▅▆▆▅▄▅▆▄▆▆▅▄▅▅▆▄▅▅▅▄▅ █
  496 μs        Histogram: log(frequency) by time       1.63 ms <

 Memory estimate: 15.88 KiB, allocs estimate: 2.

Since 1000 $\mu s$ (microseconds) = 1 $ms$ (millisecond), elapsed time for `A * B * x` is about 30 times bigger than `A * (B * x)` given problem size $n=1000$.  
Notice that memory allocation, other than flop counts, is also a criteria for computational complexity of algorithm. `A * B * x` requires about 500 times larger memory allocation than `A * (B * x)`.  
In short, `A * (B * x)` uses less resource (time and memory) than `A * B * x`  

Why are there the difference?
- $(AB)x=A(Bx)$ by associative law. For computation, $(AB)x$ involves one matrix-matrix multiplication and one matrix-vector multiplication while $A(Bx)$ involves two matrix-vector multiplications.
- Flop count : $O(n^3)$ vs $O(n^2)\quad / \quad $  Memory allocation : $O(n^2)$ vs $O(n)$
- Mathematical expression $ABx$ may suggest `A * B * x`, but in actual practice, `A * (B * x)` is a better choice for the evaluation in numerical sense.

## Performance of computer systems

* **FLOPS**. 

* For example, my laptop has the Intel i5-8279U (Coffee Lake) CPU with 4 cores runing at 2.4 GHz (cycles per second).

In [3]:
versioninfo()
# core : the number of CPU. 4cores computer can simultaneously compute four operations.
# clock speed : computer computes one operation per each cycle of clock. 3.2GHZ means clock bounces 3.2 * 10^9 cycles per second.
# word-size : 64bit computer can deal with 64 bit object at once during computation.


Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


Intel Coffee Lake CPUs can do 16 double-precision flops per cycle and 32 single-precision flops per cycle. Then the **theoretical throughput** of my laptop is
$$ 4 \times 2.4 \times 10^9 \times 16 = 153.6  \text{ GFLOPS} $$
in double precision and
$$ 4 \times 2.4 \times 10^9 \times 32 = 307.2  \text{ GFLOPS} $$
in single precision.  
* Terraflops is equal to 1000 GFLOPS(gigaflops). Thus my desktop has $1/6$ speed of terraflops supercomputer.

* In Julia, `LinearAlgebra.peakflops()` computes the peak flop rate of the computer by using `gemm!` (double precision  matrix-matrix multiplication, which is well optimized routine).

In [4]:
using LinearAlgebra

LinearAlgebra.peakflops(2^14) # matrix size 2^14

# CPU를 학대함으로써 퍼포먼스 최대치(peakflop)를 끌어내보자는 것

6.686463987238992e10

which is about 147.4 GFLOPS DP.

## Stability of numerical algorithms

* Recall that abstractly, a *problem* can be viewed as function $f: \mathcal{X} \to \mathcal{Y}$ where $\mathcal{X}$ is a normed vector space of data and $\mathcal{Y}$ is a normed vector space of solutions.
\
For given data $x \in \mathcal{X}$, the true solution of the problem $f$ is $y = f(x) \in \mathcal{Y}$. 
    - The problem of solving $Ax=b$ for fixed $b$ is $f$ defined by $ A \mapsto A^{-1}b$ with $\mathcal{X}=\{M\in\mathbb{R}^{n\times n}: M \text{ is invertible} \}$ and $\mathcal{Y} = \mathbb{R}^n$.

    
* An *algorithm* can be viewed as another map $\tilde{f}: \mathcal{X} \to \mathcal{Y}$ (algorithm approximates $f$ by $\tilde{f}$ )
    * This is because of some constraint : finite precision of computer, efficiency and design of algorithm, etc.
    * Stability can be a criteria for good algorithm in aspect of accuracy other than complexity  
\
For given data $x \in \mathcal{X}$, the solution **computed** by algorithm $\tilde{f}$ is $\hat{y} = \tilde{f}(x) \in \mathcal{Y}$. 
    - Example 1: solve $Ax=b$ by GEPP followed by forward and backward substitutions on a digital computer. ($\hat y_1= \tilde{f}_1(x)$)
    - Example 2: solve $Ax=b$ by Gauss-Seidel (an iterative method to come) on a digital computer.  ($\hat y_2= \tilde{f}_2(x)$)
    - In both cases, the solutions (in $\mathcal{Y}$) are not the same as $A^{-1}b$! ($y$ need not be same with $\hat y_1$ or $\hat y_2$)
        + We'll learn about these algorithms soon.
    - Algorithms will be affected by at least rounding errors.
    
* It is not necessarily true that $\hat{y} = y$, or $\tilde{f}(x) = f(x)$. The forward error of a computed solution is the relative error
$$
    \frac{\Vert \tilde{f}(x) - f(x) \Vert}{\Vert f(x) \Vert}
    .
$$
We want that the forward error should be bounded.
    
* Algorithm $\tilde{f}$ is said *stable* if
$$
    \forall x \in \mathcal{X}, 
    \exists \tilde{x} \in \mathcal{X} \text{ such that }
    \frac{\|\tilde{x} - x\|}{\|x\|} = O(\epsilon)
    \implies
    \frac{\|\tilde{f}(x) - f(\tilde{x})\|}{\|f(\tilde{x})\|} = O(\epsilon)
    ,
    \quad
    \text{as}~ \epsilon \to 0
    .
$$
In words, a stable algorithm gives "nearly the right" answer to a "slightly wrong" question.  
$\tilde{x}$ : a slightly wrong question. $\quad f(\tilde{x})$ : the right answer to a sligtly wrong question. $\quad \tilde{f}(x)$ : what an algorithm gives  
Note that the definition of *stable algorithm* is differed from the concept of stability which is related to bounded forward error.

* Backward stability: algorithm $\tilde{f}$ is said *backward stable* if
$$
    \forall x \in \mathcal{X}, 
    \exists \tilde{x} \in \mathcal{X} \text{ such that }
    \frac{\|\tilde{x} - x\|}{\|x\|} = O(\epsilon)
    \implies
    \tilde{f}(x) = f(\tilde{x})
    ,
    \quad
    \text{as}~ \epsilon \to 0
$$
In words, a backward stable algorithm gives "exactly the right" answer to a "slightly wrong" question.
    - Backward stability implies stability, but not the other way around.

* If a backward stable algorithm $\tilde{f}$ is applied to solve a problem $f$, the forward error of $\tilde{f}$ is bounded by the condition number of problem $f$. 
\
\
To see this, recall the definition of the condition number 
$$
    \kappa = \lim_{\delta\to 0}\sup_{\|\tilde{x} - x\|\le \delta \Vert x \Vert}\frac{\|f(\tilde{x}) - f(x)\|/\|f(x)\|}{\|\tilde{x} - x\|/\|x\|}
    .
$$
Thus for $\tilde{x} \in \mathcal{X}$ such that $\frac{\Vert\tilde{x} - x\Vert}{\Vert x \Vert} = O(\epsilon)$ and $\tilde{f}(x) = f(\tilde{x})$  as $\epsilon \to 0$, we have
$$
    \frac{\|\tilde{f}(x) - f(x)\|}{\|f(x)\|} \le ( \kappa + o(1) )\frac{\|\tilde{x} - x\|}{\|x\|}
    = O(\kappa \epsilon)
    .
$$
$$
\begin{align*}
    \because \frac{\|\tilde{f}(x) - f(x)\|}{\|f(x)\|} &=\frac{\|f(\tilde{x}) - f(x)\|}{\|f(x)\|} \\ &\leq \frac{\|\tilde{x}-x\|}{\|x\|}\sup_{\|\hat{x} - x\|\le \delta \Vert x \Vert}\frac{\|f(\hat{x}) - f(x)\|/\|f(x)\|}{\|\hat{x} - x\|/\|x\|} \\
    As\; \delta\rightarrow 0, \; \frac{\|\tilde{f}(x) - f(x)\|}{\|f(x)\|} &\leq \frac{\|\tilde{x}-x\|}{\|x\|} \kappa = O(\epsilon)\kappa=O(\kappa \epsilon)  
\end{align*}
$$


* Examples
    - Computing the inner product $x^Ty$ of vectors $x$ and $y$ using by $[\sum_{i=1}^n [[x_i][y_i]]]$ (in IEEE754) is backward stable.
    - Computing the outer product $A=xy^T$ of vectors $x$ and $y$ using by $A_{ij}=[[x_i][y_j]]$ (in IEEE754) is  stable but *not* backward stable.
    
* **(Backward) Stability is a property of an algorithm, whereas conditioning is a property of a problem (and an input data).**
    - Consider solving $Ax=b$ for fixed $b$. Then *conditioning* focuses on "how singular input $A$ is", while *stability* deals with that given $A$ and $b$, "how stable the solution derived by an algorithm is"   

## Reading

[What is Numerical Stability?](https://nhigham.com/2020/08/04/what-is-numerical-stability/) by Nick Higham

## Acknowledgment

This lecture note has evolved from [Dr. Hua Zhou](http://hua-zhou.github.io)'s 2019 Spring Statistical Computing course notes available at <http://hua-zhou.github.io/teaching/biostatm280-2019spring/index.html>.